In [1]:
import pandas as pd
import numpy as np

In [2]:
categorical = pd.read_csv('./files_for_lab/categorical.csv')
numerical = pd.read_csv('./files_for_lab/numerical.csv')
targets = pd.read_csv('./files_for_lab/target.csv')

In [7]:
data = pd.concat([numerical, categorical, targets], axis = 1)
data['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [11]:
data = data[data['TARGET_B'] == 1]

In [12]:
data['TARGET_B'].value_counts()

1    4843
Name: TARGET_B, dtype: int64

In [14]:
numerical = data.select_dtypes(include=['int', 'float'])
categorical = data.select_dtypes(include=['object'])


In [17]:
y.shape

(4843,)

In [18]:
X.shape

(4843, 339)

FEATURES SELECTION

In [20]:
scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(numerical)

In [21]:
from sklearn.feature_selection import VarianceThreshold 
var_threshold = 0.02
sel = VarianceThreshold(threshold=(var_threshold))

sel = sel.fit(numerical_scaled)
temp = sel.transform(numerical_scaled)
temp = pd.DataFrame(temp)
print(numerical_scaled.shape)
print(temp.shape)

(4843, 332)
(4843, 101)


In [22]:
sel.variances_ > var_threshold
sel.get_support()
var_list = list(sel.get_support())
var_list

[False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 True

In [23]:
list(zip(numerical.columns, var_list))

[('TCODE', False),
 ('AGE', True),
 ('INCOME', True),
 ('WEALTH1', True),
 ('HIT', False),
 ('MALEMILI', False),
 ('MALEVET', True),
 ('VIETVETS', True),
 ('WWIIVETS', True),
 ('LOCALGOV', False),
 ('STATEGOV', False),
 ('FEDGOV', False),
 ('WEALTH2', True),
 ('POP901', False),
 ('POP902', False),
 ('POP903', False),
 ('POP90C1', True),
 ('POP90C2', True),
 ('POP90C3', True),
 ('POP90C4', False),
 ('POP90C5', False),
 ('ETH1', True),
 ('ETH2', True),
 ('ETH3', False),
 ('ETH4', False),
 ('ETH5', False),
 ('ETH6', False),
 ('ETH7', False),
 ('ETH8', False),
 ('ETH9', False),
 ('ETH10', False),
 ('ETH11', False),
 ('ETH12', False),
 ('ETH13', False),
 ('ETH14', False),
 ('ETH15', False),
 ('ETH16', False),
 ('AGE901', False),
 ('AGE902', False),
 ('AGE903', False),
 ('AGE904', False),
 ('AGE905', False),
 ('AGE906', False),
 ('AGE907', False),
 ('CHIL1', False),
 ('CHIL2', False),
 ('CHIL3', False),
 ('AGEC1', False),
 ('AGEC2', False),
 ('AGEC3', False),
 ('AGEC4', False),
 ('AGEC5', Fa

In [24]:
cols_to_remove = [col[0] for col in zip(numerical.columns, var_list) if col[1] == False]

In [25]:
numerical.drop(cols_to_remove, axis=1, inplace=True)

In [26]:
correlation_matrix = numerical.corr()

In [27]:
columns_to_drop = []
n_features = len(numerical.columns)

for i in range(n_features):
    for j in range(i + 1, n_features):
        if abs(correlation_matrix.iloc[i, j]) > 0.9:
            column_i = numerical.columns[i]
            column_j = numerical.columns[j]

            if column_i not in columns_to_drop:
                columns_to_drop.append(column_j)

In [28]:
numerical.drop(columns_to_drop, axis=1, inplace=True)

In [29]:
numerical.shape

(4843, 86)

In [33]:
from sklearn import linear_model
lm = linear_model.LinearRegression()

In [31]:
X = pd.concat([numerical, categorical], axis = 1)
y = data['TARGET_D']

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

numericalX_train   = X_train.select_dtypes(np.number)
numericalX_test    = X_test.select_dtypes(np.number)
categoricalX_train = X_train.select_dtypes(object) 
categoricalX_test  = X_test.select_dtypes(object) 

# we OneHotEncode the categoricals so we can use the same dataset to perform a regression later (in the lab).
# it is not needed for a DecisionTree or RandomForest model
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categoricalX_train)
encoded_categorical_train = encoder.transform(categoricalX_train).toarray()
encoded_categorical_train = pd.DataFrame(encoded_categorical_train, columns=encoder.get_feature_names_out()) # needed to avoid error
encoded_categorical_test = encoder.transform(categoricalX_test).toarray()
encoded_categorical_test = pd.DataFrame(encoded_categorical_test, columns=encoder.get_feature_names_out())

# we scale the numericals so we can use the same dataset to perform a regression later (in the lab).
# it is not needed for a DecisionTree or RandomForest model
from sklearn.preprocessing import StandardScaler
transformer = MinMaxScaler().fit(numericalX_train)
scaled_numerical_train = transformer.transform(numericalX_train)
scaled_numerical_train = pd.DataFrame(scaled_numerical_train, columns=numericalX_train.columns)
scaled_numerical_test  = transformer.transform(numericalX_test)
scaled_numerical_test  = pd.DataFrame(scaled_numerical_test, columns=numericalX_train.columns)

X_train_treated = pd.concat([scaled_numerical_train, encoded_categorical_train], axis = 1)
X_test_treated  = pd.concat([scaled_numerical_test,  encoded_categorical_test],  axis = 1)

In [47]:
lm.fit(X_train_treated,y_train)

LinearRegression()

In [48]:
from sklearn.metrics import r2_score

predictions = lm.predict(X_train_treated)
r2_score(y_train, predictions)

0.39863411723621356

In [49]:
y_train

88982    20.0
24612    20.0
62247    25.0
17784    10.0
18435    25.0
         ... 
20647    25.0
64107    30.0
32452    19.0
51732    25.0
53887    10.0
Name: TARGET_D, Length: 3874, dtype: float64

In [50]:
predictions

array([12.61748216, 16.79463481, 32.05251769, ..., 18.5347984 ,
       18.48013876, 17.66263785])

The results with linear regression models are not very good, going to compare more models

In [51]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor  # Import DecisionTreeRegressor
from sklearn.metrics import r2_score

def auto_test_models(models, X_train, X_test, y_train, y_test):
    for model in models:
        regressor = model
        regressor.fit(X_train, y_train)
        predictions = regressor.predict(X_test) 
        r2 = r2_score(y_test, predictions)
        print(str(model) + ' R2 score:', r2)

models = [LinearRegression(), KNeighborsRegressor(n_neighbors=5), DecisionTreeRegressor(random_state=0)]
auto_test_models(models, X_train_treated, X_test_treated, y_train, y_test)



LinearRegression() R2 score: 0.352865150877105
KNeighborsRegressor() R2 score: 0.1561830056904968
DecisionTreeRegressor(random_state=0) R2 score: -0.5040564869978992


Now, I am going to run the model on the predicted donors

In [55]:
%store -r donors_index
print(donors_index)

Int64Index([    3,     6,     8,    10,    11,    13,    18,    19,    22,
               25,
            ...
            76308, 76312, 76315, 76316, 76321, 76323, 76324, 76326, 76327,
            76328],
           dtype='int64', length=38243)


In [56]:
donors = data.loc[data.index.isin(donors_index)]

In [58]:
donors.head(100)

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
20,2,62.000000,3,8,10,2,25,40,27,11,...,88,1,94,4,96,3,87,1,1,4.0
30,0,61.611649,5,9,0,1,37,58,16,8,...,90,4,93,1,95,12,90,4,1,7.0
45,0,66.000000,5,9,5,0,33,24,39,6,...,93,12,94,4,96,2,87,4,1,5.0
78,0,69.000000,6,9,0,0,34,20,54,2,...,90,1,95,3,95,11,90,1,1,13.0
93,1,73.000000,1,7,10,0,21,53,8,5,...,92,9,95,9,95,9,92,9,1,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2704,0,61.611649,5,9,0,1,32,38,30,6,...,95,6,89,12,95,12,89,2,1,10.0
2712,1,79.000000,5,9,0,0,37,26,48,4,...,91,12,95,11,95,11,86,12,1,10.0
2788,0,28.000000,4,4,0,0,28,38,38,11,...,95,2,95,12,96,3,95,2,1,9.0
2848,0,45.000000,5,9,0,15,33,19,14,11,...,95,11,95,12,96,3,92,1,1,7.0


In [59]:
donors['TARGET_B'].value_counts()

1    1803
Name: TARGET_B, dtype: int64

Now, I am going to apply the model to the predicted donors

In [60]:
data_with_pred = pd.read_csv('data_with_pred.csv')

In [63]:
data_with_pred[['TARGET_B', 'pred']].value_counts()

TARGET_B  pred
0         0       54273
          1       36296
1         0        2896
          1        1947
dtype: int64